데이터 출처 : AI Hub  
https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=102

In [1]:
import pandas as pd
import numpy as np

In [34]:
work_dir = '/Users/ho/Documents/LLD/AI/소상공인 고객 주문 질의-응답 텍스트/Training/라벨링데이터_train/'

df = pd.read_csv(work_dir + '음식점_train.csv')
df

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트,가격,수량,크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
0,1,c,"참치타다끼샐러드는 취소했었는데, 맞죠?",음식점,32365,q,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,참치타다끼샐러드,32365,1
1,2,s,"네, 참치타다끼샐러드는 취소된 상태입니다.",음식점,32365,a,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,참치타다끼샐러드,32365,2
2,3,c,주문한 메뉴 취소할 때 주문서만 있으면 된다는 말씀이세요?,음식점,15016,q,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15016,1
3,4,s,"네, 주문서만 있으면 취소 가능합니다.",음식점,15016,a,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15016,2
4,5,c,고객서비스가 엉망이군요. 그냥 갑니다.,음식점,62984,q,n,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62984,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290382,290383,s,성북동까지 지금 시간에는 40분 이상 걸립니다.,음식점,50614,a,m,배송_날짜_요청,NaN,NaN,NaN,성북동,NaN,NaN,NaN,NaN,NaN,50614,2
290383,290384,c,짜장면 지금 주문하면 바로 배달 올 수 있죠?,음식점,18579,q,m,배송_날짜_요청,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,짜장면,18579,1
290384,290385,s,"네, 지금 주문하시면 바로 배달 갑니다.",음식점,18579,a,m,배송_날짜_요청,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18579,2
290385,290386,c,롯데유치원에 피자 20판 보내야하는데 5시까지 될까요?,음식점,59570,q,m,배송_날짜_요청,NaN,NaN,NaN,롯데유치원,NaN,NaN,5시,NaN,피자,59570,1


In [5]:
# 필요한 컬럼만 추출
df = df[['발화자','발화문']]
df['label'] = None

# c와 s가 상호 대화한 내역만 추출
for i in range(len(df)-1):
    if df.loc[i, '발화자'] == 'c' and df.loc[i+1, '발화자'] == 's':
        df.loc[i, 'label'] = df.loc[i+1, '발화문']

df = df.dropna(axis=0)
df = df.drop(['발화자'], axis=1)
df = df.reset_index(drop=True)

# 필요한 이름으로 컬렴명 변경
df.columns = ['prompt', 'completion']

df

,prompt,completion
0,"참치타다끼샐러드는 취소했었는데, 맞죠?","네, 참치타다끼샐러드는 취소된 상태입니다."
1,주문한 메뉴 취소할 때 주문서만 있으면 된다는 말씀이세요?,"네, 주문서만 있으면 취소 가능합니다."
2,고객서비스가 엉망이군요. 그냥 갑니다.,원하시는 서비스를 제공하지못해 죄송합니다.
3,방금 전에 홍합탕 주문한 거 지금 취소 가능한 거 맞죠?,"네, 주문하신 홍합탕은 아직 조리 전이라 취소 가능합니다."
4,"매운무뼈닭발 롯데주유소로 주문한 거 아직 안받았는데, 취소해도 되는 거죠?","아니요, 이미 접수 후 조리 시작하여 취소는 어렵습니다."
...,...,...
117522,짜장면 12:30까지 배달되야 먹는데 그 시간까지 오실 수 있나요?,"네, 말씀하신 시간까지 배달해 드릴게요."
117523,후라이드 5마리 시킨 곳인데 배달 출발한 거 확실해요?,"네, 배달 출발한 건지 알아 보고 연락 드리겠습니다."
117524,성북동까지 치킨 배달시키면 얼마나 걸리는지 알려주세요.,성북동까지 지금 시간에는 40분 이상 걸립니다.
117525,짜장면 지금 주문하면 바로 배달 올 수 있죠?,"네, 지금 주문하시면 바로 배달 갑니다."


In [31]:
work_dir = '/Users/ho/Documents/LLD/AI/소상공인 고객 주문 질의-응답 텍스트/Training/라벨링데이터_train/'
categories = [
    '음식점', 
    '패션', 
    '카페', 
    '출산육아', 
    '의류', 
    '뷰티', 
    '기타', 
    '식품', 
    '슈퍼', 
    '생활잡화', 
    '병원', 
    '디지털가전', 
    '건강', 
    '가구인테리어'
]

In [33]:
# 카테고리 별 데이터 핸들링
for ct in categories:
    df = pd.read_csv(work_dir + ct + '_train.csv', dtype=str)
    df = df[['발화자','발화문']]

    df['label'] = None

    for i in range(len(df)-1):
        if df.loc[i, '발화자'] == 'c' and df.loc[i+1, '발화자'] == 's':
            df.loc[i, 'label'] = df.loc[i+1, '발화문']

    df = df.dropna(axis=0)
    df = df.drop(['발화자'], axis=1)
    df = df.reset_index(drop=True)
    df.columns = ['prompt', 'completion']
    
    # 핸들링한 df를 json 파일로 저장
    json = df.to_json(
        '/Users/ho/Documents/LLD/AI/' + ct + '_data.json', 
        orient='records', 
        indent = 4, 
        force_ascii=False
    )
    
    print(ct + ' complet!')

음식점 complet!
패션 complet!
카페 complet!
출산육아 complet!
의류 complet!
뷰티 complet!
기타 complet!
식품 complet!
슈퍼 complet!
생활잡화 complet!
병원 complet!
디지털가전 complet!
건강 complet!
가구인테리어 complet!
